In [93]:
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage, bpm2tempo, tempo2bpm
from midi_player import MIDIPlayer
import os

In [95]:
bpm2tempo(120)

500000

In [41]:
data_path = '_adl-piano-midi/midi/adl-piano-midi'
total_midi_count = 0
for path in os.listdir(data_path):
    midi_count = 0
    for _, _, files in os.walk(os.path.join(data_path, path)):
        for file in files:
            if file.endswith('.mid'): midi_count += 1
    print(path, midi_count)
    total_midi_count += midi_count

print('Total:', total_midi_count)

Pop 750
.DS_Store 0
Blues 129
Latin 253
Ambient 185
World 503
Reggae 26
Electronic 161
Classical 1398
Rock 3137
Religious 55
Folk 90
Unknown 2171
Soul 351
Children 24
Soundtracks 947
Country 246
Rap 158
Jazz 492
Total: 11076


In [42]:
# TODO: Create the midi parser.
# Following https://arxiv.org/pdf/1808.03715 - 5.0.1

In [220]:
# midi_path = './_adl-piano-midi/midi/adl-piano-midi/Classical/Classical Flute/James Galway/Hero.mid'
# midi_path = './_adl-piano-midi/midi/adl-piano-midi/Folk/Alpine Yodeling/Jacob Sisters/La-La-La.mid'
midi_path = './_google_maestro_v3/2004/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi'

In [221]:
mid = mido.MidiFile(midi_path)
print(len(mid.tracks))
for track in mid.tracks:
    for msg in track:
        if msg.type in ['note_on']:
            print('\t', msg)
        else:
            print(msg)

2
MetaMessage('set_tempo', tempo=500000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('end_of_track', time=1)
program_change channel=0 program=0 time=0
control_change channel=0 control=64 value=110 time=0
control_change channel=0 control=67 value=117 time=0
control_change channel=0 control=64 value=114 time=490
control_change channel=0 control=64 value=118 time=94
control_change channel=0 control=67 value=121 time=75
control_change channel=0 control=64 value=122 time=262
	 note_on channel=0 note=71 velocity=60 time=128
control_change channel=0 control=67 value=117 time=37
control_change channel=0 control=67 value=113 time=55
	 note_on channel=0 note=71 velocity=0 time=1
control_change channel=0 control=64 value=111 time=41
control_change channel=0 control=64 value=93 time=19
control_change channel=0 control=64 value=76 time=17
	 note_on channel=0 note=55 velocity=44 time=9
	 note_on channel=0 n

In [222]:
def get_instructions():
    pass

In [ ]:
def midi_to_tensor(midi_path):

    mid = mido.MidiFile(midi_path)
    instruction_library = {
        'time_shift': lambda x: ('time_shift', x),
        'note_on': lambda x: ('note_on', x),
        'note_off': lambda x: ('note_off', x),
        'velocity': lambda x: ('velocity', x)
    }

    midi_instructions = []
    tempo = 0
    sustain_threshold = 100
    
    for track in mid.tracks:

        sustain = False
        sustained_notes = set()
        last_velocity = -1
        time_delta = 0

        for msg in track:

            # Set the tempo in the first instance of a tempo message
            if msg.is_meta:
                if (msg.type == "set_tempo") and (tempo == 0):
                    tempo = msg.tempo
                continue

            if msg.time > 0:
                time_delta += msg.time

            # Note On
            if (msg.type == 'note_on') and (msg.velocity != 0):

                # Time Shift
                if time_delta > 0:
                    midi_instructions.append(instruction_library['time_shift'](time_delta))
                    time_delta = 0

                if sustain and (msg.note in sustained_notes):
                    midi_instructions.append(instruction_library['note_off'](msg.note))

                # Change velocity if there is a bucket change
                velocity_bucket = msg.velocity // 4 # 0-31
                if last_velocity != velocity_bucket:
                    midi_instructions.append(instruction_library['velocity'](velocity_bucket))
                    last_velocity = velocity_bucket
                    
                midi_instructions.append(instruction_library['note_on'](msg.note))

            # Note Off
            elif (msg.type == 'note_off') or ((msg.type == 'note_on') and (msg.velocity == 0)):

                # Time Shift
                if time_delta > 0:
                    midi_instructions.append(instruction_library['time_shift'](time_delta))
                    time_delta = 0

                if sustain:
                    sustained_notes.add(msg.note)
                else:
                    midi_instructions.append(instruction_library['note_off'](msg.note))

            # Sustain Pedal
            elif (msg.type == 'control_change') and (msg.control == 64):

                if msg.value >= sustain_threshold: # Sustain pedal on                  
                    sustain = True
                elif sustain and (msg.value < sustain_threshold): # Sustain pedal off

                    # Time Shift
                    if time_delta > 0:
                        midi_instructions.append(instruction_library['time_shift'](time_delta))
                        time_delta = 0

                    sustain = False
                    # Turn sustained_notes off
                    for sustained_note in sustained_notes:
                        midi_instructions.append(instruction_library['note_off'](sustained_note))
                    sustained_notes.clear()
                else:
                    continue

            else:
                continue

    return midi_instructions, tempo

In [233]:
test_midi_path = 'test.mid'
def tensor_to_midi(midi_instructions, tempo, save_path=None):
    midi_out = MidiFile()
    # midi_out.tracks.append(mid.tracks[0])

    track = MidiTrack()
    track.append(MetaMessage('set_tempo', tempo=tempo))

    time_shift = 0
    d_velocity = 0
    for instruction in midi_instructions:
        if instruction[0] == 'time_shift':
            time_shift += instruction[1]
        elif instruction[0] == 'velocity':
            d_velocity = instruction[1]
        else:
            track.append(Message(instruction[0], note=instruction[1], velocity=d_velocity*4, time=time_shift))
            time_shift = 0

    track.append(MetaMessage('end_of_track', time=1))
    midi_out.tracks.append(track)

    if save_path:
        midi_out.save(save_path)

    return midi_out

In [234]:
midi_instructions, tempo = midi_to_tensor(midi_path)
midi_out = tensor_to_midi(midi_instructions, tempo, test_midi_path)

In [235]:
MIDIPlayer(test_midi_path, 400)

In [231]:
MIDIPlayer(midi_path, 400)